# Introduction

## What is Mathematical Programming and Combinatorial Optimization?

A mathematical program is an optimization problem of the form:

$\max \{ f(x) : x \in X, \; g(x) \le 0, \; h(x) = 0 \}$

where $\textstyle X$ is a subset of $\mathbb{R}^n$ and is in the domain of $f$,  $g$ and $h$, which map into real spaces. The relations,  $x \in X$, $g(x) \le 0$, and $h(x) = 0$ are called constraints, and $f$ is called the objective function.

## How Do We Solve These Types of Problems?

A *lot* of research has been done to develop advanced solvers that can take problem definitions in a canonical form and produce optimal solutions. So the good news is, you don't need to create the solver! The bad news is, you need to figure out how to set up your problem and write it down in a way these solvers can understand.

## Python and Pyomo To The Rescue! 

[Pyomo](https://www.pyomo.org/) is a generalized mathematical programming modeling language in Python that gives you a wonderful set of abstractions and the full power of the Python programming language for setting up optimization problems. Once you've set up your problem in Pyomo, it does the work of: 

  1. Applying any necessary transformations to your problem definition to bring it into a canonical form that solvers can handle
  2. Writing out that canonicalized form into the appropriate input file syntax for your solver of choice
  3. Monitoring the solver and giving you useful updates as it runs
  4. Parsing the solver outputs into a well-structured Python object that you can interrogate and otherwise manipulate for your purposes

# Example Problem: Machine Bottleneck

**Our Task:** Given a set of $M$ generic machines that can process any job from a set of $J$ jobs with a given job release time, job duration, and job due date for each job, find the schedule that optimizes our objective function (Spoiler: We can choose multiple different objective functions, and each will yield different schedule results!)

## Imports 

In [ ]:
%matplotlib widget

In [ ]:
import itertools
import os
import json
import copy
import pprint
import random
import functools
import collections

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyomo.environ as pyo
import pyomo.gdp as pygdp

from IPython.display import display, JSON, Markdown, display_javascript, display_html, YouTubeVideo

## Utility Functions and Problem Setup

In [ ]:
def schedule_machines_bigM(JOBS, MACHINES):
    
    # create model
    m = pyo.ConcreteModel()
    
    # index set to simplify notation
    m.J = pyo.Set(initialize=JOBS.keys())
    m.M = pyo.Set(initialize=MACHINES)
    m.PAIRS = pyo.Set(initialize = m.J * m.J, dimen=2, filter=lambda m, j, k : j < k)

    # decision variables
    m.start      = pyo.Var(m.J, bounds=(0, 1000))
    m.makespan   = pyo.Var(domain=pyo.NonNegativeReals)
    m.pastdue    = pyo.Var(m.J, domain=pyo.NonNegativeReals)
    m.early      = pyo.Var(m.J, domain=pyo.NonNegativeReals)
    
    # additional decision variables for use in the objecive
    m.ispastdue  = Var(m.J, domain=pyo.Binary)
    m.maxpastdue = Var(domain=pyo.NonNegativeReals)
    
    # for binary assignment of jobs to machines
    m.z = pyo.Var(m.J, m.M, domain=pyo.Binary)

    # for modeling disjunctive constraints
    m.y = pyo.Var(m.PAIRS, domain=pyo.Binary)
    BigM = max([JOBS[j]['release'] for j in m.J]) + sum([JOBS[j]['duration'] for j in m.J])

    m.OBJ = pyo.Objective(expr = sum(m.pastdue[j] for j in m.J) + m.makespan - sum(m.early[j] for j in m.J), sense = pyo.minimize)

    # Job must start after its release date
    m.c1 = pyo.Constraint(m.J, rule=lambda m, j: 
            m.start[j] >= JOBS[j]['release'])
    # TODO: IDK 
    m.c2 = pyo.Constraint(m.J, rule=lambda m, j:
            m.start[j] + JOBS[j]['duration'] + m.early[j] == JOBS[j]['due'] + m.pastdue[j])
    # 
    m.c3 = pyo.Constraint(m.J, rule=lambda m, j: 
            sum(m.z[j,mach] for mach in m.M) == 1)
    m.c4 = pyo.Constraint(m.J, rule=lambda m, j: 
            m.pastdue[j] <= BigM*m.ispastdue[j])
    m.c5 = pyo.Constraint(m.J, rule=lambda m, j: 
            m.pastdue[j] <= m.maxpastdue)
    m.c6 = pyo.Constraint(m.J, rule=lambda m, j: 
            m.start[j] + JOBS[j]['duration'] <= m.makespan)
    m.d1 = pyo.Constraint(m.M, m.PAIRS, rule = lambda m, mach, j, k:
            m.start[j] + JOBS[j]['duration'] <= m.start[k] + BigM*(m.y[j,k] + (1-m.z[j,mach]) + (1-m.z[k,mach])))
    m.d2 = pyo.Constraint(m.M, m.PAIRS, rule = lambda m, mach, j, k: 
            m.start[k] + JOBS[k]['duration'] <= m.start[j] + BigM*((1-m.y[j,k]) + (1-m.z[j,mach]) + (1-m.z[k,mach])))
    
    return m

def schedule_machines_disjunct(JOBS, MACHINES, transform="gdp.hull", objective=minimal_pastdue_plus_makespan_minus_early):
    
    # create model
    m = pyo.ConcreteModel()
    
    ##############
    # SETS
    ##############
    # index set to simplify notation
    m.J = pyo.Set(initialize=JOBS.keys())
    m.M = pyo.Set(initialize=MACHINES)
    m.PAIRS = pyo.Set(initialize = m.J * m.J, dimen=2, filter=lambda m, j, k : j < k)
    
    ##############
    # Parameters
    ##############
    m.release = pyo.Param(JOBS.keys(), initialize=lambda m, j: JOBS[j]['release'])
    m.duration = pyo.Param(JOBS.keys(), initialize=lambda m, j: JOBS[j]['duration'])
    m.due = pyo.Param(JOBS.keys(), initialize=lambda m, j: JOBS[j]['due'])

    ##############
    # Variables
    ##############
    m.start      = pyo.Var(m.J, bounds=(0, 1000))
    m.makespan   = pyo.Var(domain=pyo.NonNegativeReals)
    m.pastdue    = pyo.Var(m.J, bounds=(0, 1000))
    m.early      = pyo.Var(m.J, bounds=(0, 10000))
    
    # additional decision variables for use in the objecive
    m.ispastdue  = pyo.Var(m.J, domain=pyo.Binary)
    m.maxpastdue = pyo.Var(domain=pyo.NonNegativeReals)
    
    # for binary assignment of jobs to machines
    m.z = pyo.Var(m.J, m.M, domain=pyo.Binary)

    ##############
    # Objective
    ##############
    m.OBJ = objective(m)

    ##############
    # Constraints
    ##############
    
    # job starts after it is released
    m.c1 = pyo.Constraint(m.J, rule=lambda m, j: m.start[j] >= JOBS[j]['release'])

    # defines early and pastdue
    m.c2 = pyo.Constraint(m.J, rule=lambda m, j: m.start[j] + JOBS[j]['duration'] + m.early[j] == JOBS[j]['due'] + m.pastdue[j])
    m.d1 = pygdp.Disjunction(m.J, rule=lambda m, j: [m.early[j]==0, m.pastdue[j]==0])

    # each job is assigned to one and only one machine
    m.c3 = pyo.Constraint(m.J, rule=lambda m, j: sum(m.z[j, mach] for mach in m.M) == 1)

    # defines a binary variable indicating if a job is past due
    m.c4 = pygdp.Disjunction(m.J, rule=lambda m, j: [m.pastdue[j] == 0, m.ispastdue[j] == 1])

    # all jobs must be finished before max pastdue
    m.c5 = pyo.Constraint(m.J, rule=lambda m, j: m.pastdue[j] <= m.maxpastdue)

    # defining make span
    m.c6 = pyo.Constraint(m.J, rule=lambda m, j: m.start[j] + JOBS[j]['duration'] <= m.makespan)
 
    # disjunctions
    def no_collision_rule(disjunct, machine, job1, job2, indicator): 
        """
        """
        model = disjunct.model()
        disjunct.constraints = pyo.ConstraintList()
        end_of_job1 = model.start[job1] + JOBS[job1]['duration']
        end_of_job2 = model.start[job2] + JOBS[job2]['duration']
        start_of_job1 = model.start[job1]
        start_of_job2 = model.start[job2]
        if indicator == 0:
            # machines must be the same
            disjunct.constraints.add(model.z[job1, machine] == model.z[job2, machine])
            # job1 must end before job2 starts
            disjunct.constraints.add(end_of_job1 <= start_of_job2)
        elif indicator == 1:
            # machines must be the same
            disjunct.constraints.add(model.z[job1, machine] == model.z[job2, machine])
            # job2 must end before job1 starts
            disjunct.constraints.add(end_of_job2 <= start_of_job1)
        else:
            # the jobs are on different machines
            disjunct.constraints.add(model.z[job1, machine] + model.z[job2, machine] <= 1)
        
    m.NoCollision = pygdp.Disjunct(m.M * m.PAIRS, [0, 1, 2, 3, 4], rule=no_collision_rule)
    def disj_rule(model, mach, job1, job2):
        """
        """
        return [model.NoCollision[mach, job1, job2, indicator] for indicator in [0, 1, 2, 3, 4]]
    m.no_collision_disjunction = pygdp.Disjunction(m.M * m.PAIRS, rule=disj_rule)
  
    print("Applying transform {} ...".format(transform))
    transform = pyo.TransformationFactory(transform)
    transform.apply_to(m)
    return m
   
def solve(solver, model, jobs, machines, tee=True):
    print("Running model solve ...")
    solver.solve(model, tee=tee).write()
    print("Setting up schedule dict ...")
    schedule = {}
    for j in model.J:
        schedule[j] = {
            'start': model.start[j](), 
            'finish': model.start[j]() + jobs[j]['duration'],
            'machine': [mach for mach in machines if int(model.z[j,mach].value) == 1][0]
        }
    return model, schedule

def gantt(JOBS, SCHEDULE={}, ax=None):
    bw = 0.3
    if ax:
        fig = ax.get_figure()
    else:
        fig, ax = plt.subplots(figsize=(12, 0.7*(len(JOBS.keys()))))
    idx = 0
    for j, data in sorted(SCHEDULE.items()):
        x = JOBS[j]['release']
        y = JOBS[j]['due']
        plt.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='cyan', alpha=0.6)
        if j in SCHEDULE.keys():
            x = SCHEDULE[j]['start']
            y = SCHEDULE[j]['finish']
            plt.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='red', alpha=0.5)
            plt.plot([x,y,y,x,x], [idx-bw,idx-bw,idx+bw,idx+bw,idx-bw],color='k')
            if "id" in data:
                short_name = data['id'][0:5] + "_" + data['id'].split("_")[-1]
            else:
                short_name = str(j)
            plt.text((SCHEDULE[j]['start'] + SCHEDULE[j]['finish'])/2.0,idx,
                'Job\n' + short_name, color='black', weight='bold',
                horizontalalignment='center', verticalalignment='center')
        idx += 1

    plt.ylim(-0.5, idx-0.5)
    plt.title('Job Schedule')
    plt.xlabel('Time')
    plt.ylabel('Jobs')
    plt.yticks(range(len(JOBS)), JOBS.keys())
    plt.grid()
    xlim = plt.xlim()
    
    if SCHEDULE:
        for j in SCHEDULE.keys():
            if 'machine' not in SCHEDULE[j].keys():
                SCHEDULE[j]['machine'] = 1
        MACHINES = sorted(set([SCHEDULE[j]['machine'] for j in SCHEDULE.keys()]))

        plt.figure(figsize=(12, 0.8*len(MACHINES)))
        for j, data in sorted(SCHEDULE.items()):
            idx = MACHINES.index(SCHEDULE[j]['machine'])
            x = SCHEDULE[j]['start']
            y = SCHEDULE[j]['finish']
            plt.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='red', alpha=0.5)
            plt.plot([x,y,y,x,x], [idx-bw,idx-bw,idx+bw,idx+bw,idx-bw],color='k')
            if "id" in data:
                short_name = data['id'][0:5] + "_" + data['id'].split("_")[-1]
            else:
                short_name = str(j)
            plt.text((SCHEDULE[j]['start'] + SCHEDULE[j]['finish'])/2.0, idx,
                'Job\n' + short_name, color='black', weight='bold',
                horizontalalignment='center', verticalalignment='center')
        plt.xlim(xlim)
        plt.ylim(-0.5, len(MACHINES)-0.5)
        plt.title('Machine Schedule')
        plt.yticks(range(len(MACHINES)), MACHINES)
        plt.ylabel('Machines')
        plt.grid()
    return fig, ax
        
def kpi(JOBS, SCHEDULE):
    KPI = {}
    KPI['Makespan'] = max(SCHEDULE[job]['finish'] for job in SCHEDULE)
    KPI['Max Pastdue'] = max(max(0, SCHEDULE[job]['finish'] - JOBS[job]['due']) for job in SCHEDULE)
    KPI['Sum of Pastdue'] = sum(max(0, SCHEDULE[job]['finish'] - JOBS[job]['due']) for job in SCHEDULE)
    KPI['Number Pastdue'] = sum(SCHEDULE[job]['finish'] > JOBS[job]['due'] for job in SCHEDULE)
    KPI['Number on Time'] = sum(SCHEDULE[job]['finish'] <= JOBS[job]['due'] for job in SCHEDULE)
    KPI['Fraction on Time'] = KPI['Number on Time']/len(SCHEDULE)
    return KPI

## Test Data

In [1]:
test_machines = ['M-A','M-B']

test_jobs = {
    'A': {'release': 2, 'duration': 5, 'due': 10},
    'B': {'release': 5, 'duration': 6, 'due': 21},
    'C': {'release': 4, 'duration': 8, 'due': 15},
    'D': {'release': 0, 'duration': 4, 'due': 10},
    'E': {'release': 0, 'duration': 2, 'due':  5},
    'F': {'release': 8, 'duration': 3, 'due': 15},
    'G': {'release': 9, 'duration': 2, 'due': 22},
    'H': {'release': 6, 'duration': 3, 'due': 22},
}

## Minimal Makespan

In [ ]:
def minimal_makespan(model):
    return pyo.Objective(
        expr = model.makespan,
        sense = pyo.minimize
    )

model = schedule_machines_disjunct(test_jobs, test_machines, objective=minimal_makespan)
solver = pyo.SolverFactory("cbc")
solver.options["threads"] = 4 # this adds additional threads to the solver to speed things up, adjust accordingly
solver.options['Seconds'] = 300 # this sets a hard timelimit on the solver
model, SCHEDULE = solve(solver, model, test_jobs, test_machines, tee=True) 
gantt(test_jobs, SCHEDULE)
kpi(test_jobs, SCHEDULE)

## Minimal Past Due

In [ ]:
def minimal_pastdue(model):
    return pyo.Objective(
        expr = sum(model.pastdue[j] for j in model.J),
        sense = pyo.minimize
    )

model = schedule_machines_disjunct(test_jobs, test_machines, objective=minimal_pastdue)
solver = pyo.SolverFactory("cbc")
solver.options["threads"] = 4
solver.options['Seconds'] = 300 # this sets a hard timelimit
model, SCHEDULE = solve(solver, model, test_jobs, test_machines, tee=True) 
gantt(test_jobs, SCHEDULE)
kpi(test_jobs, SCHEDULE)

## Minimal Makespan and Past Due

In [ ]:
def minimal_pastdue_plus_makespan(model):
    return pyo.Objective(
        expr = sum(model.pastdue[j] for j in model.J) + model.makespan,
        sense = pyo.minimize
    )

model = schedule_machines_disjunct(test_jobs, test_machines, objective=minimal_pastdue_plus_makespan)
solver = pyo.SolverFactory("cbc")
solver.options["threads"] = 4
solver.options['Seconds'] = 300 # this sets a hard timelimit
model, SCHEDULE = solve(solver, model, test_jobs, test_machines, tee=True) 
gantt(test_jobs, SCHEDULE)
kpi(test_jobs, SCHEDULE)

## Minimal Makespan and Past Due, Maximal Earliness

In [ ]:
def minimal_pastdue_plus_makespan_minus_early(model):
    return pyo.Objective(
        expr = sum(model.pastdue[j] for j in model.J) + model.makespan - sum(model.early[j] for j in model.J),
        sense = pyo.minimize
    )

def minimal_pastdue_plus_makespan_plus_delay(model):
    return pyo.Objective(
        expr = sum(model.pastdue[j] for j in model.J) + model.makespan + sum(model.start[j] - model.release[j] for j in model.J),
        sense = pyo.minimize
    )

model = schedule_machines_disjunct(test_jobs, test_machines, objective=minimal_pastdue_plus_makespan_plus_delay)
solver = pyo.SolverFactory("cbc")
solver.options["threads"] = 4
solver.options['Seconds'] = 300 # this sets a hard timelimit
model, SCHEDULE = solve(solver, model, test_jobs, test_machines, tee=True) 
gantt(test_jobs, SCHEDULE)
kpi(test_jobs, SCHEDULE)

# What Else Can We Do With Pyomo and Mathematical Programming?

So much! Check out this talk! 

In [ ]:
YouTubeVideo("oivo-XAvum8")